In [1]:
from google.colab import files
uploaded = files.upload()

zip_name = None
for k in uploaded.keys():
    if k.endswith('.zip'):
        zip_name = k
        break
assert zip_name is not None, 'Please upload agentic-forecasting.zip'
print('Uploaded:', zip_name)

Saving agentic-forecasting.zip to agentic-forecasting.zip
Uploaded: agentic-forecasting.zip


In [2]:
!unzip -o agentic-forecasting.zip -d /content
%cd /content/agentic-forecasting
!python -m pip install --upgrade pip
!pip install -r requirements.txt

Archive:  agentic-forecasting.zip
   creating: /content/agentic-forecasting/
  inflating: /content/agentic-forecasting/run_demo.py  
  inflating: /content/agentic-forecasting/.DS_Store  
  inflating: /content/__MACOSX/agentic-forecasting/._.DS_Store  
  inflating: /content/agentic-forecasting/requirements.txt  
   creating: /content/agentic-forecasting/agentic_forecasting/
  inflating: /content/agentic-forecasting/README.md  
   creating: /content/agentic-forecasting/configs/
  inflating: /content/agentic-forecasting/agentic_forecasting/.DS_Store  
  inflating: /content/__MACOSX/agentic-forecasting/agentic_forecasting/._.DS_Store  
  inflating: /content/agentic-forecasting/agentic_forecasting/config.py  
  inflating: /content/agentic-forecasting/agentic_forecasting/graph.py  
  inflating: /content/agentic-forecasting/agentic_forecasting/__init__.py  
  inflating: /content/agentic-forecasting/agentic_forecasting/features.py  
  inflating: /content/agentic-forecasting/agentic_forecasting

In [3]:
enable_macro = True  #@param {type:'boolean'}
enable_garch = True  #@param {type:'boolean'}
fred_api_key = "f8c1d22bf3c7cb7aa4bb532a5efd81cc"  #@param {type:'string'}

import yaml
cfg_path = 'configs/app.yaml'
with open(cfg_path) as f:
    cfg = yaml.safe_load(f)
cfg['use_macro'] = bool(enable_macro)
cfg['use_garch'] = bool(enable_garch)
cfg['fred_api_key'] = fred_api_key or None
with open(cfg_path, 'w') as f:
    yaml.safe_dump(cfg, f)
print('Updated config:', cfg)

Updated config: {'assets': ['AAPL', 'BTC/USDT'], 'horizons': ['1d', '7d'], 'use_sentiment': False, 'use_macro': True, 'use_garch': True, 'transaction_costs_bps': 2, 'slippage_bps': 10, 'fred_api_key': 'f8c1d22bf3c7cb7aa4bb532a5efd81cc'}


In [4]:
use_prophet_ensemble = True  #@param {type:'boolean'}

if use_prophet_ensemble:
    # Read graph.py and replace node_forecast with an ensemble version
    import re, io
    from pathlib import Path
    p = Path('agentic_forecasting/graph.py')
    src = p.read_text()
    # Simple replacement: define a new node_forecast using ARIMA + Prophet
    ensemble_code = '''\
def node_forecast(state: PipeState)->PipeState:
    from .forecasting import walk_forward_arima, walk_forward_prophet
    close = state["ohlcv"]["Close"]
    wf1 = walk_forward_arima(close, order=(2,1,2))
    try:
        wf2 = walk_forward_prophet(close)
        # Equal weights; adjust if desired
        pred = (wf1.predictions.reindex(close.index).fillna(0) + wf2.predictions.reindex(close.index).fillna(0)) / 2.0
    except Exception:
        # Prophet may not be installed on some platforms; fallback to ARIMA only
        pred = wf1.predictions.reindex(close.index)
    return {**state, "forecast": pred}
'''
    # Replace the original node_forecast block by regex
    src = re.sub(r"def node_forecast\(state: PipeState\)->PipeState:[\s\S]*?\n\ndef node_strategy", ensemble_code + "\n\ndef node_strategy", src)
    p.write_text(src)
    print('graph.py updated with ARIMA+Prophet ensemble.')
else:
    print('Leaving node_forecast as ARIMA baseline.')

graph.py updated with ARIMA+Prophet ensemble.


In [9]:
from google.colab import files
uploaded = files.upload()  # select af_merge_hotfix.zip

Saving af_merge_hotfix.zip to af_merge_hotfix.zip


In [10]:
!unzip -o af_merge_hotfix.zip

Archive:  af_merge_hotfix.zip
  inflating: af_merge_hotfix.py      


In [12]:
from af_merge_hotfix import ensure_datetime_index, ensure_flat_columns, safe_time_merge

In [14]:
import pandas as pd

def _safe_time_merge(left, right, on=None, how='left', **kwargs):
    # Flatten any MultiIndex columns
    L = ensure_flat_columns(left)
    R = ensure_flat_columns(right)

    if on is None:
        # Merge on index: ensure both indexes are DatetimeIndex named 'datetime'
        L = ensure_datetime_index(L)
        R = ensure_datetime_index(R)
        return L.merge(R, left_index=True, right_index=True, how=how, **{k:v for k,v in kwargs.items() if k not in ('on','left_index','right_index')})

    # Merge on a column
    if on not in L.columns and on in getattr(L.index, "names", []):
        L = L.reset_index()
    if on not in R.columns and on in getattr(R.index, "names", []):
        R = R.reset_index()

    # Be forgiving about types for the key
    for df in (L, R):
        if on in df.columns:
            try:
                df[on] = pd.to_datetime(df[on], errors='ignore')
            except Exception:
                pass

    return pd.merge(L, R, on=on, how=how, **kwargs)

In [15]:
from pandas.errors import MergeError as _PandasMergeError

_orig_df_merge = pd.DataFrame.merge

def _df_merge_safe(self, right, *args, **kwargs):
    try:
        return _orig_df_merge(self, right, *args, **kwargs)
    except _PandasMergeError as e:
        if "Not allowed to merge between different levels" in str(e):
            # Try rescuing with our safe time merge
            on = kwargs.get("on", None)
            how = kwargs.get("how", "left")
            return _safe_time_merge(self, right, on=on, how=how, **{k:v for k,v in kwargs.items() if k not in ("on","how")})
        raise

pd.DataFrame.merge = _df_merge_safe

In [17]:
import pandas as pd
import numpy as np

def _ensure_flat_columns(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df = df.copy()
        df.columns = ["_".join(map(str, lev)) for lev in df.columns.values]
        return df
    if any(isinstance(c, tuple) for c in df.columns):
        df = df.copy()
        df.columns = ["_".join(map(str, c)) if isinstance(c, tuple) else str(c) for c in df.columns]
    return df

def _ensure_datetime_index(df: pd.DataFrame, index_name: str = "datetime") -> pd.DataFrame:
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Expected DataFrame")
    out = df.copy()

    # Already datetime index
    if isinstance(out.index, pd.DatetimeIndex):
        out.index = pd.DatetimeIndex(out.index)
        out.index.name = index_name
        return out

    # If MultiIndex, try to pull a datetime-like level
    if isinstance(out.index, pd.MultiIndex):
        for cand in [index_name, "date", "time", "timestamp"]:
            if cand in out.index.names:
                out.index = pd.DatetimeIndex(out.index.get_level_values(cand))
                out.index.name = index_name
                return out
        for lvl in range(out.index.nlevels):
            try:
                dt = pd.to_datetime(out.index.get_level_values(lvl), errors="raise")
                out.index = pd.DatetimeIndex(dt)
                out.index.name = index_name
                return out
            except Exception:
                pass
        out = out.reset_index()

    # Look for datetime-ish columns
    for c in out.columns:
        if np.issubdtype(out[c].dtype, np.datetime64):
            out = out.set_index(c)
            out.index = pd.DatetimeIndex(out.index)
            out.index.name = index_name
            return out

    for cand in [index_name, "date", "time", "timestamp"]:
        if cand in out.columns:
            out[cand] = pd.to_datetime(out[cand], errors="coerce")
            if out[cand].notna().any():
                out = out.set_index(cand)
                out.index.name = index_name
                return out

    # Last attempt: parse existing index
    try:
        out.index = pd.to_datetime(out.index, errors="raise")
        out.index.name = index_name
        return out
    except Exception:
        pass

    raise ValueError("Could not establish a DatetimeIndex; provide a 'datetime' column.")

In [18]:
from pandas.errors import MergeError as _PandasMergeError

def _safe_time_merge(left, right, on=None, how="left", **kwargs):
    L = _ensure_flat_columns(left)
    R = _ensure_flat_columns(right)

    if on is None:
        L = _ensure_datetime_index(L)
        R = _ensure_datetime_index(R)
        return L.merge(R, left_index=True, right_index=True, how=how, **{k:v for k,v in kwargs.items() if k not in ("on","left_index","right_index")})

    # Merge on a column
    if on not in L.columns and on in getattr(L.index, "names", []):
        L = L.reset_index()
    if on not in R.columns and on in getattr(R.index, "names", []):
        R = R.reset_index()

    # Be forgiving on key dtype
    for df in (L, R):
        if on in df.columns:
            try:
                df[on] = pd.to_datetime(df[on], errors="ignore")
            except Exception:
                pass

    return pd.merge(L, R, on=on, how=how, **kwargs)

In [19]:
# Patch BOTH pd.merge and DataFrame.merge to auto-rescue this exact error
_orig_pd_merge = pd.merge
_orig_df_merge = pd.DataFrame.merge

def _pd_merge_safe(left, right, *args, **kwargs):
    try:
        return _orig_pd_merge(left, right, *args, **kwargs)
    except _PandasMergeError as e:
        if "Not allowed to merge between different levels" in str(e):
            return _safe_time_merge(left, right, **kwargs)
        raise

def _df_merge_safe(self, right, *args, **kwargs):
    try:
        return _orig_df_merge(self, right, *args, **kwargs)
    except _PandasMergeError as e:
        if "Not allowed to merge between different levels" in str(e):
            return _safe_time_merge(self, right, **kwargs)
        raise

pd.merge = _pd_merge_safe
pd.DataFrame.merge = _df_merge_safe
print("✅ Merge monkey-patch active.")

✅ Merge monkey-patch active.


In [20]:
graph = af.build_graph()
out_aapl = graph.invoke({"asset": "AAPL"})

MergeError: Not allowed to merge between different levels. (2 levels on the left, 1 on the right)